In [1]:
import pandas as pd
import numpy as np

In [2]:
visa = pd.read_csv("data/visa.csv", index_col = 0)

visa.groupby("citizenship")["decision"].value_counts()

citizenship  decision
AFGHANISTAN  Y            3
             N            1
ALBANIA      Y           10
             N            2
ALGERIA      N            2
                         ..
YUGOSLAVIA   N            1
ZAMBIA       Y            3
             N            1
ZIMBABWE     Y           16
             N            1
Name: decision, Length: 277, dtype: int64

In [115]:
visa["citizenship"].value_counts() / len(visa)

INDIA          0.601591
CHINA          0.055116
CANADA         0.042316
SOUTH KOREA    0.041616
PHILIPPINES    0.027364
                 ...   
GABON          0.000026
MALTA          0.000026
SUDAN          0.000026
TOGO           0.000026
MACAU          0.000026
Name: citizenship, Length: 161, dtype: float64

In [3]:
visa.head()

,category,citizenship,application type,visa class,decision
0,Automotive,ECUADOR,MAILEDIN,EWI,Y
1,Advanced Mfg,INDIA,ONLINE,H-1B,Y
2,Advanced Mfg,PHILIPPINES,ONLINE,L-1,Y
3,Advanced Mfg,INDIA,ONLINE,H-1B,Y
4,Advanced Mfg,INDIA,ONLINE,H-1B,Y


In [4]:
visa["visa class"].unique()

array(['EWI', 'H-1B', 'L-1', 'E-2', 'TPS', 'B-2', 'B-1', 'F-1', 'H-2B',
       'VWT', 'Parolee', 'TN', 'Not in USA', 'H-4', 'F-2', 'E-3', 'O-1',
       'H-2A', 'H-1B1', 'E-1', 'G-5', 'A-3', 'A1/A2', 'H-3', 'P-1', 'C-1',
       'L-2', 'VWB', 'J-1', 'O-3', 'O-2', 'J-2', 'G-4', 'H-1A', 'R-1',
       'I', 'H1B', 'N', 'T-1', 'TD', 'Parol', 'AOS/H-1B', 'M-1', 'P-3',
       'K-1'], dtype=object)

- H-1B: work visa for professionals working in US
- L-1: work visa for foreiners opening business in US
- F-1: student visa
- Parolee: a permit for non-US national who does not have valid immigrant visa, but need to reenter

In [70]:
((visa["visa class"].value_counts() / len(visa)).head(4)).to_json()

'{"H-1B":0.85157161,"L-1":0.0362001503,"F-1":0.0284781426,"Parolee":0.0195382343}'

In [77]:
(((visa["visa class"].value_counts() / len(visa)).head(4))
 .reset_index()
 .rename(columns={"index":"name","visa class":"y"})
).to_json(orient="records")

'[{"name":"H-1B","y":0.85157161},{"name":"L-1","y":0.0362001503},{"name":"F-1","y":0.0284781426},{"name":"Parolee","y":0.0195382343}]'

In [80]:
1 - sum((visa["visa class"].value_counts() / len(visa)).head(4))

0.06421186286958092

In [88]:
(visa["visa class"].value_counts() / len(visa)).head(4).index

Index(['H-1B', 'L-1', 'F-1', 'Parolee'], dtype='object')

In [102]:
visa[visa["visa class"].isin(['H-1B', 'L-1', 'F-1', 'Parolee'])].head()

,category,citizenship,application type,visa class,decision
1,Advanced Mfg,INDIA,ONLINE,H-1B,Y
2,Advanced Mfg,PHILIPPINES,ONLINE,L-1,Y
3,Advanced Mfg,INDIA,ONLINE,H-1B,Y
4,Advanced Mfg,INDIA,ONLINE,H-1B,Y
5,Advanced Mfg,INDIA,ONLINE,H-1B,Y


In [106]:
visa.loc[~visa["visa class"].isin(['H-1B', 'L-1', 'F-1', 'Parolee']),"visa class"] = "Other"

In [107]:
visa["visa class"].value_counts() / len(visa)

H-1B       0.851572
Other      0.064212
L-1        0.036200
F-1        0.028478
Parolee    0.019538
Name: visa class, dtype: float64

In [113]:
visa_class = (pd.pivot_table(
    (visa.groupby("visa class")["decision"]
    .value_counts()
    .to_frame()
    .rename(columns={"decision":"count"})), 
    index=["visa class"], 
    columns=["decision"],
    values="count")
 .fillna(0)
).rename(columns={"N":"n", "Y":"y"})
visa_class["percent"] = visa_class["y"] / (visa_class["n"] + visa_class["y"])
# visa_decision.to_csv("data/visa_decision.csv")
visa_class["percent"].to_json(orient="records")

'[0.7306642402,0.8495268235,0.8625626342,0.6719128329,0.848806366]'

In [7]:
visa.columns = visa.columns.str.lower()
visa.to_csv("./data/visa.csv")
visa.head()

,category,citizenship,application type,visa class,decision
0,Automotive,ECUADOR,MAILEDIN,EWI,Y
1,Advanced Mfg,INDIA,ONLINE,H-1B,Y
2,Advanced Mfg,PHILIPPINES,ONLINE,L-1,Y
3,Advanced Mfg,INDIA,ONLINE,H-1B,Y
4,Advanced Mfg,INDIA,ONLINE,H-1B,Y


In [83]:
visa_decision = (pd.pivot_table(
    (visa.groupby("citizenship")["decision"]
    .value_counts()
    .to_frame()
    .rename(columns={"decision":"count"})), 
    index=["citizenship"], 
    columns=["decision"],
    values="count")
 .fillna(0)
).rename(columns={"N":"n", "Y":"y"})
visa_decision.head()

decision,n,y
citizenship,,
AFGHANISTAN,1.0,3.0
ALBANIA,2.0,10.0
ALGERIA,2.0,1.0
ARGENTINA,22.0,67.0
ARMENIA,0.0,9.0


In [10]:
visa_decision["percent"] = visa_decision["y"] / (visa_decision["n"] + visa_decision["y"])
# visa_decision.to_csv("data/visa_decision.csv")
visa_decision

decision,n,y,percent
citizenship,,,
AFGHANISTAN,1.0,3.0,0.750000
ALBANIA,2.0,10.0,0.833333
ALGERIA,2.0,1.0,0.333333
ARGENTINA,22.0,67.0,0.752809
ARMENIA,0.0,9.0,1.000000
...,...,...,...
VIETNAM,9.0,55.0,0.859375
YEMEN,0.0,3.0,1.000000
YUGOSLAVIA,1.0,2.0,0.666667


In [57]:
job_pro = (visa[visa["decision"]=="Y"]["category"].value_counts() / len(visa) * 100).to_frame()
job_pro.head()

,category
IT,35.484958
Advanced Mfg,12.951206
Other Economic Sector,9.445207
Finance,6.193154
Educational Services,5.350989


In [58]:
job_pro = (visa[visa["decision"]=="N"]["category"].value_counts() / len(visa) * 100).to_frame()
job_pro.head()

,category
IT,7.429193
Advanced Mfg,2.220725
Other Economic Sector,2.117074
Educational Services,1.088337
Health Care,0.847348


In [61]:
visa["decision"].value_counts() / len(visa)

Y    0.835195
N    0.164805
Name: decision, dtype: float64

In [52]:
job_pro = (visa["category"].value_counts() / len(visa) * 100).to_frame()
job_pro.head()

,category
IT,42.914151
Advanced Mfg,15.171931
Other Economic Sector,11.562281
Finance,6.957581
Educational Services,6.439325


In [53]:
job_pro.to_json()

'{"category":{"IT":42.9141509678,"Advanced Mfg":15.1719312793,"Other Economic Sector":11.5622813609,"Finance":6.9575807831,"Educational Services":6.4393252313,"Health Care":5.1929206292,"Retail":3.3168355316,"Aerospace":3.0447513669,"Hospitality":1.2956388795,"Automotive":0.9095384934,"Energy":0.8525303827,"Construction":0.8136612163,"Transportation":0.6789147729,"Biotechnology":0.355005053,"Agribusiness":0.3213184421,"Geospatial":0.1269726102,"Homeland Security":0.0466429997}}'

In [54]:
job_pro = job_pro.reset_index().rename(columns={"index":"name","category":"y"})
job_pro.head()

,name,y
0,IT,42.914151
1,Advanced Mfg,15.171931
2,Other Economic Sector,11.562281
3,Finance,6.957581
4,Educational Services,6.439325


In [56]:
job_pro.to_json(orient='records')

'[{"name":"IT","y":42.9141509678},{"name":"Advanced Mfg","y":15.1719312793},{"name":"Other Economic Sector","y":11.5622813609},{"name":"Finance","y":6.9575807831},{"name":"Educational Services","y":6.4393252313},{"name":"Health Care","y":5.1929206292},{"name":"Retail","y":3.3168355316},{"name":"Aerospace","y":3.0447513669},{"name":"Hospitality","y":1.2956388795},{"name":"Automotive","y":0.9095384934},{"name":"Energy","y":0.8525303827},{"name":"Construction","y":0.8136612163},{"name":"Transportation","y":0.6789147729},{"name":"Biotechnology","y":0.355005053},{"name":"Agribusiness","y":0.3213184421},{"name":"Geospatial","y":0.1269726102},{"name":"Homeland Security","y":0.0466429997}]'

In [17]:
job_pro.to_csv("data/job_pro.csv", index=False)

In [13]:
pd.read_csv("data/visa_decision.csv")

,Unnamed: 0,N,Y,percent
0,af,1.0,3.0,0.750000
1,al,2.0,10.0,0.833333
2,dz,2.0,1.0,0.333333
3,ar,22.0,67.0,0.752809
4,am,0.0,9.0,1.000000
...,...,...,...,...
154,ve,43.0,219.0,0.835878
155,vn,9.0,55.0,0.859375
156,ye,0.0,3.0,1.000000
157,zm,1.0,3.0,0.750000
